In [7]:
import pandas as pd
import numpy as np
import requests
import json
from api_keys import GOOGLE_API_KEY, POSTGRES_PWD
import time
import dill

In [8]:
with open('../pkl/00_cleaned_city_names.pkl', 'r') as fh:
    location_key = dill.load(fh)

In [9]:
location_key.head(2)

,location,location_type,country,province,county,city
0,Argentina-Buenos_Aires,province,Argentina,Buenos Aires,None,None
1,Argentina-CABA,province,Argentina,Ciudad de Buenos Aires,None,None


In [10]:
def get_latitude_longitude(df_row):
    subtype = df_row.location_type
    second_str = df_row[subtype].replace(' ', '+')
    country_str = df_row.country.replace(' ', '+')
    
    url = 'https://maps.googleapis.com/maps/api/geocode/json?address=+{},+{}&key={}'.format(second_str, 
                                                                                            country_str, 
                                                                                            GOOGLE_API_KEY)
    
    try:
        response = requests.get(url).text
        lat_lng = json.loads(response)['results'][0]['geometry']['location']
        lat_lng_df = pd.Series({'lat':lat_lng['lat'], 'lng':lat_lng['lng']})
    except:
        lat_lng_df = pd.Series({'lat':np.NaN, 'lng':np.NaN})
        
    time.sleep(1)
    return lat_lng_df

In [ ]:
location_key[['latitude','longitude']] = location_key.apply(lambda x: get_latitude_longitude(x), axis=1)

In [ ]:
location_key_out = location_key[['location','latitude','longitude']]

In [ ]:
location_key_out.to_csv('../csv/01_latitude_longitude.csv', sep=',', index=False)

In [ ]:
with open('../pkl/01_latitude_longitude.pkl', 'w') as fh:
    dill.dump(location_key_out, fh)

In [ ]:
from api_keys import GOOGLE_API_KEY, POSTGRES_PWD

In [ ]:
postgres_str = 'postgresql://mlgill:{}@127.0.0.1:12345/mcnulty'.format(POSTGRES_PWD)
engine = create_engine(postgres_str)
location_key.to_sql('01_latitude_longitude', engine, if_exists='replace')